# ICU Vital Sign Forecasting - Data Preprocessing & Feature Engineering

**Project:** Forecasting ICU Patient Vital-Sign Deterioration Using Hybrid Statistical and Deep Learning Models

**This Notebook Covers:**
1. Data Cleaning (outlier removal, validation)
2. Temporal Resampling (irregular → regular intervals)
3. Missing Value Imputation
4. Normalization/Standardization
5. Feature Engineering (rolling stats, rate of change)
6. Windowed Sequence Creation (input → forecast)
7. Train/Validation/Test Split
8. Deterioration Label Creation

---

## 1. Setup and Load Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.3f}'.format)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Libraries imported successfully")

Libraries imported successfully


In [57]:
## INSERT PATH FROM YOUR DESTINATION
BASE_PATH = r"C:\Users\hasin\Downloads\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4"

# If you saved the extracted vitals from EDA notebook:
VITALS_PATH = f"{BASE_PATH}\\vitals_sample_2000.csv"
ICUSTAYS_PATH = f"{BASE_PATH}\\ICUSTAYS.csv\\ICUSTAYS.csv"
ADMISSIONS_PATH = f"{BASE_PATH}\\ADMISSIONS.csv\\ADMISSIONS.csv"
PATIENTS_PATH = f"{BASE_PATH}\\PATIENTS.csv\\PATIENTS.csv"

# Output paths
OUTPUT_PATH = f"{BASE_PATH}\\processed"

# Create output directory
import os
os.makedirs(OUTPUT_PATH, exist_ok=True)

print("Paths configured")

Paths configured


In [17]:
# Load data
print("Loading data...")

# Load vitals (from EDA notebook output or reload from CHARTEVENTS)
try:
    vitals_df = pd.read_csv(VITALS_PATH)
    print(f"Vitals loaded from saved file: {len(vitals_df):,} records")
except FileNotFoundError:
    print("Saved vitals file not found. Please run the EDA notebook first to extract vitals.")

# Load supporting tables
icustays = pd.read_csv(ICUSTAYS_PATH)
admissions = pd.read_csv(ADMISSIONS_PATH)
patients = pd.read_csv(PATIENTS_PATH)

print(f"ICUSTAYS: {len(icustays):,} rows")
print(f"ADMISSIONS: {len(admissions):,} rows")
print(f"PATIENTS: {len(patients):,} rows")

Loading data...
Vitals loaded from saved file: 1,663,885 records
ICUSTAYS: 61,532 rows
ADMISSIONS: 58,976 rows
PATIENTS: 46,520 rows


In [18]:
# Convert datetime columns
vitals_df['CHARTTIME'] = pd.to_datetime(vitals_df['CHARTTIME'])
icustays['INTIME'] = pd.to_datetime(icustays['INTIME'])
icustays['OUTTIME'] = pd.to_datetime(icustays['OUTTIME'])
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'])

print("Datetime columns converted")
print(f"\nVitals date range: {vitals_df['CHARTTIME'].min()} to {vitals_df['CHARTTIME'].max()}")

Datetime columns converted

Vitals date range: 2100-08-08 08:30:00 to 2205-10-15 22:00:00


## 2. Data Cleaning

In [19]:
# Define ITEMID mappings (from EDA)
VITAL_ITEMIDS = {
    'HR': [211, 220045],
    'SpO2': [646, 220277],
    'SBP': [51, 442, 455, 220179, 220050],
    'DBP': [8368, 8440, 8441, 220180, 220051],
    'MBP': [456, 52, 6702, 220052, 220181],
    'RR': [618, 615, 220210, 224690],
    'Temp_C': [223761, 678],
    'Temp_F': [223762, 679]
}

# Physiologically plausible ranges
VITAL_RANGES = {
    'HR': (20, 300),
    'SpO2': (50, 100),
    'SBP': (40, 300),
    'DBP': (20, 200),
    'MBP': (30, 250),
    'RR': (4, 60),
    'Temp_C': (30, 45),
    'Temp_F': (86, 113)
}

# Map ITEMIDs to vital names
def map_vital_name(itemid):
    for vital_name, itemids in VITAL_ITEMIDS.items():
        if itemid in itemids:
            return vital_name
    return 'Other'

# Add vital type column if not present
if 'VITAL_TYPE' not in vitals_df.columns:
    vitals_df['VITAL_TYPE'] = vitals_df['ITEMID'].apply(map_vital_name)

print("Vital types assigned:")
print(vitals_df['VITAL_TYPE'].value_counts())

Vital types assigned:
VITAL_TYPE
HR        335685
RR        268656
SBP       248313
DBP       247554
MBP       247252
SpO2      241829
Temp_C     53495
Temp_F     21101
Name: count, dtype: int64


In [20]:
# Clean data: Remove physiologically implausible values
print("="*60)
print("DATA CLEANING")
print("="*60)

# Convert VALUENUM to numeric
vitals_df['VALUENUM'] = pd.to_numeric(vitals_df['VALUENUM'], errors='coerce')

initial_count = len(vitals_df)
print(f"\nInitial records: {initial_count:,}")

# Remove null values
vitals_df = vitals_df.dropna(subset=['VALUENUM'])
print(f"After removing nulls: {len(vitals_df):,} ({initial_count - len(vitals_df):,} removed)")

# Remove out-of-range values
def remove_outliers(df):
    clean_df = df.copy()
    removed_counts = {}
    
    for vital, (low, high) in VITAL_RANGES.items():
        mask = clean_df['VITAL_TYPE'] == vital
        before = mask.sum()
        
        # Mark invalid values
        invalid_mask = mask & ((clean_df['VALUENUM'] < low) | (clean_df['VALUENUM'] > high))
        removed_counts[vital] = invalid_mask.sum()
        
        # Remove invalid
        clean_df = clean_df[~invalid_mask]
    
    return clean_df, removed_counts

vitals_clean, removed = remove_outliers(vitals_df)

print("\nOutliers removed by vital type:")
for vital, count in removed.items():
    if count > 0:
        print(f"  {vital}: {count:,} records")

print(f"\nFinal clean records: {len(vitals_clean):,}")
vitals_df = vitals_clean.copy()

DATA CLEANING

Initial records: 1,663,885
After removing nulls: 1,662,064 (1,821 removed)

Outliers removed by vital type:
  HR: 91 records
  SpO2: 153 records
  SBP: 578 records
  DBP: 665 records
  MBP: 653 records
  RR: 1,486 records
  Temp_C: 53,359 records
  Temp_F: 3,767 records

Final clean records: 1,601,312


In [7]:
# Combine Temperature (C and F) into single variable
print("\n--- Combining Temperature Variables ---")

# Convert Fahrenheit to Celsius
temp_f_mask = vitals_df['VITAL_TYPE'] == 'Temp_F'
vitals_df.loc[temp_f_mask, 'VALUENUM'] = (vitals_df.loc[temp_f_mask, 'VALUENUM'] - 32) * 5/9
vitals_df.loc[temp_f_mask, 'VITAL_TYPE'] = 'Temp'

# Rename Temp_C to Temp
vitals_df.loc[vitals_df['VITAL_TYPE'] == 'Temp_C', 'VITAL_TYPE'] = 'Temp'

print("Temperature unified (all in Celsius)")
print(vitals_df['VITAL_TYPE'].value_counts())


--- Combining Temperature Variables ---
Temperature unified (all in Celsius)
VITAL_TYPE
HR      335532
RR      266961
SBP     247103
DBP     246889
MBP     246047
SpO2    241576
Temp     17204
Name: count, dtype: int64


## 3. Temporal Resampling (Hourly Aggregation)

In [21]:
print("="*60)
print("TEMPORAL RESAMPLING")
print("="*60)

# Add ICU stay information
vitals_df = vitals_df.merge(
    icustays[['ICUSTAY_ID', 'INTIME', 'OUTTIME', 'LOS']],
    on='ICUSTAY_ID',
    how='left'
)

# Calculate hours since ICU admission
vitals_df['HOURS_IN'] = (vitals_df['CHARTTIME'] - vitals_df['INTIME']).dt.total_seconds() / 3600

# Filter: Only keep measurements during ICU stay (0 to LOS hours)
vitals_df = vitals_df[(vitals_df['HOURS_IN'] >= 0) & (vitals_df['HOURS_IN'] <= vitals_df['LOS'] * 24)]

print(f"Records within ICU stay: {len(vitals_df):,}")

TEMPORAL RESAMPLING
Records within ICU stay: 1,594,852


In [22]:
# Create hour bins for resampling
vitals_df['HOUR_BIN'] = vitals_df['HOURS_IN'].apply(lambda x: int(x))

# Aggregate to hourly means
print("\nAggregating to hourly means...")

vitals_hourly = vitals_df.groupby(
    ['ICUSTAY_ID', 'SUBJECT_ID', 'HADM_ID', 'HOUR_BIN', 'VITAL_TYPE']
)['VALUENUM'].agg(['mean', 'std', 'count']).reset_index()

vitals_hourly.columns = ['ICUSTAY_ID', 'SUBJECT_ID', 'HADM_ID', 'HOUR', 'VITAL_TYPE', 'VALUE', 'STD', 'COUNT']

print(f"Hourly aggregated records: {len(vitals_hourly):,}")


Aggregating to hourly means...
Hourly aggregated records: 1,266,615


In [23]:
# Pivot to wide format (one column per vital)
print("\nPivoting to wide format...")

vitals_wide = vitals_hourly.pivot_table(
    index=['ICUSTAY_ID', 'SUBJECT_ID', 'HADM_ID', 'HOUR'],
    columns='VITAL_TYPE',
    values='VALUE',
    aggfunc='mean'
).reset_index()

# Flatten column names
vitals_wide.columns.name = None

print(f"Wide format shape: {vitals_wide.shape}")
print(f"Columns: {list(vitals_wide.columns)}")
vitals_wide.head()


Pivoting to wide format...
Wide format shape: (286288, 12)
Columns: ['ICUSTAY_ID', 'SUBJECT_ID', 'HADM_ID', 'HOUR', 'DBP', 'HR', 'MBP', 'RR', 'SBP', 'SpO2', 'Temp_C', 'Temp_F']


,ICUSTAY_ID,SUBJECT_ID,HADM_ID,HOUR,DBP,HR,MBP,RR,SBP,SpO2,Temp_C,Temp_F
0,200006.000,10950,189514,1,87.000,81.000,97.333,21.333,118.000,100.000,NaN,NaN
1,200006.000,10950,189514,2,64.000,73.000,79.333,14.000,110.000,99.000,NaN,NaN
2,200006.000,10950,189514,3,53.000,78.000,69.000,18.000,101.000,100.000,NaN,NaN
3,200006.000,10950,189514,4,50.000,77.000,65.000,20.000,95.000,99.000,NaN,NaN
4,200006.000,10950,189514,5,58.000,80.000,76.667,24.500,114.000,97.500,NaN,NaN


In [24]:
# Create complete hourly time series for each ICU stay
print("\nCreating complete hourly grid...")

def create_complete_timeline(group):
    """Fill in missing hours for each ICU stay"""
    icustay_id = group['ICUSTAY_ID'].iloc[0]
    subject_id = group['SUBJECT_ID'].iloc[0]
    hadm_id = group['HADM_ID'].iloc[0]
    
    min_hour = int(group['HOUR'].min())
    max_hour = int(group['HOUR'].max())
    
    # Create complete hour range
    all_hours = pd.DataFrame({'HOUR': range(min_hour, max_hour + 1)})
    all_hours['ICUSTAY_ID'] = icustay_id
    all_hours['SUBJECT_ID'] = subject_id
    all_hours['HADM_ID'] = hadm_id
    
    # Merge with existing data
    complete = all_hours.merge(group, on=['ICUSTAY_ID', 'SUBJECT_ID', 'HADM_ID', 'HOUR'], how='left')
    
    return complete

# Apply to each ICU stay
vitals_complete = vitals_wide.groupby('ICUSTAY_ID', group_keys=False).apply(create_complete_timeline)
vitals_complete = vitals_complete.reset_index(drop=True)

print(f"Complete timeline shape: {vitals_complete.shape}")
print(f"Total hours: {len(vitals_complete):,}")


Creating complete hourly grid...
Complete timeline shape: (307210, 12)
Total hours: 307,210


## 4. Missing Value Imputation

In [25]:
print("="*60)
print("MISSING VALUE IMPUTATION")
print("="*60)

# Define vital columns
VITAL_COLS = ['HR', 'SpO2', 'SBP', 'DBP', 'MBP', 'RR', 'Temp']
VITAL_COLS = [col for col in VITAL_COLS if col in vitals_complete.columns]

# Check missingness before imputation
print("\nMissingness before imputation:")
for col in VITAL_COLS:
    missing = vitals_complete[col].isna().sum()
    pct = missing / len(vitals_complete) * 100
    print(f"  {col}: {missing:,} ({pct:.1f}%)")

MISSING VALUE IMPUTATION

Missingness before imputation:
  HR: 22,218 (7.2%)
  SpO2: 114,612 (37.3%)
  SBP: 113,662 (37.0%)
  DBP: 113,750 (37.0%)
  MBP: 114,663 (37.3%)
  RR: 110,774 (36.1%)


In [26]:
# Imputation strategy:
# 1. Forward fill (carry last observation forward) - up to 4 hours
# 2. Backward fill (for initial missing values) - up to 4 hours
# 3. Linear interpolation for remaining gaps - up to 6 hours
# 4. Mark longer gaps as missing (for masking during training)

MAX_FFILL_HOURS = 4
MAX_INTERP_HOURS = 6

def impute_vitals(group):
    """Impute missing values for a single ICU stay"""
    df = group.copy()
    
    for col in VITAL_COLS:
        # Forward fill (up to MAX_FFILL_HOURS)
        df[col] = df[col].ffill(limit=MAX_FFILL_HOURS)
        
        # Backward fill (up to MAX_FFILL_HOURS)
        df[col] = df[col].bfill(limit=MAX_FFILL_HOURS)
        
        # Linear interpolation (up to MAX_INTERP_HOURS)
        df[col] = df[col].interpolate(method='linear', limit=MAX_INTERP_HOURS)
    
    return df

print("Imputing missing values...")
vitals_imputed = vitals_complete.groupby('ICUSTAY_ID', group_keys=False).apply(impute_vitals)

# Check missingness after imputation
print("\nMissingness after imputation:")
for col in VITAL_COLS:
    missing = vitals_imputed[col].isna().sum()
    pct = missing / len(vitals_imputed) * 100
    print(f"  {col}: {missing:,} ({pct:.1f}%)")

Imputing missing values...

Missingness after imputation:
  HR: 1,145 (0.4%)
  SpO2: 97,072 (31.6%)
  SBP: 97,413 (31.7%)
  DBP: 97,412 (31.7%)
  MBP: 97,742 (31.8%)
  RR: 96,947 (31.6%)


In [27]:
# Create missingness mask (for model training - to mask predictions at truly missing points)
print("\nCreating missingness masks...")

for col in VITAL_COLS:
    vitals_imputed[f'{col}_MISSING'] = vitals_complete[col].isna().astype(int)

# For remaining NaN after imputation, fill with population median
for col in VITAL_COLS:
    median_val = vitals_imputed[col].median()
    vitals_imputed[col] = vitals_imputed[col].fillna(median_val)

print("All missing values imputed")


Creating missingness masks...
All missing values imputed


## 5. Normalization / Standardization

In [28]:
print("="*60)
print("NORMALIZATION")
print("="*60)

# Store original values for later inverse transform
vitals_original = vitals_imputed[VITAL_COLS].copy()

# Calculate and store normalization parameters
normalization_params = {}
for col in VITAL_COLS:
    normalization_params[col] = {
        'mean': vitals_imputed[col].mean(),
        'std': vitals_imputed[col].std(),
        'min': vitals_imputed[col].min(),
        'max': vitals_imputed[col].max()
    }

print("Normalization parameters:")
params_df = pd.DataFrame(normalization_params).T
print(params_df)

NORMALIZATION
Normalization parameters:
        mean    std    min     max
HR   107.934 35.783 20.000 254.000
SpO2  97.447  2.425 50.000 100.000
SBP  122.519 18.840 40.000 268.000
DBP   60.019 11.689 20.000 200.000
MBP   79.121 13.089 30.000 249.000
RR    20.302  4.965  4.000  60.000


In [29]:
# Apply Z-score normalization (StandardScaler)
# Better for LSTM and gradient-based methods

scaler = StandardScaler()
vitals_imputed[VITAL_COLS] = scaler.fit_transform(vitals_imputed[VITAL_COLS])

print("\nAfter Z-score normalization:")
print(vitals_imputed[VITAL_COLS].describe())


After Z-score normalization:
              HR       SpO2        SBP        DBP        MBP         RR
count 307210.000 307210.000 307210.000 307210.000 307210.000 307210.000
mean      -0.000      0.000     -0.000     -0.000     -0.000     -0.000
std        1.000      1.000      1.000      1.000      1.000      1.000
min       -2.457    -19.568     -4.380     -3.424     -3.753     -3.283
25%       -0.781     -0.184     -0.505     -0.429     -0.468     -0.464
50%       -0.334      0.228     -0.081     -0.087     -0.086     -0.061
75%        0.952      0.640      0.344      0.341      0.296      0.342
max        4.082      1.053      7.722     11.976     12.978      7.995


In [30]:
# Save scaler for later use (inverse transform predictions)
import pickle

scaler_path = f"{OUTPUT_PATH}\\vital_scaler.pkl"
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

# Also save normalization params as CSV for reference
params_df.to_csv(f"{OUTPUT_PATH}\\normalization_params.csv")

print(f"Scaler saved to: {scaler_path}")

Scaler saved to: C:\Users\hasin\Downloads\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\processed\vital_scaler.pkl


## 6. Feature Engineering

In [31]:
print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

def add_temporal_features(df):
    """Add time-based features for each ICU stay"""
    result = df.copy()
    
    for col in VITAL_COLS:
        # Rolling statistics (past 6 hours)
        result[f'{col}_roll_mean_6h'] = result.groupby('ICUSTAY_ID')[col].transform(
            lambda x: x.rolling(window=6, min_periods=1).mean()
        )
        result[f'{col}_roll_std_6h'] = result.groupby('ICUSTAY_ID')[col].transform(
            lambda x: x.rolling(window=6, min_periods=1).std()
        )
        result[f'{col}_roll_min_6h'] = result.groupby('ICUSTAY_ID')[col].transform(
            lambda x: x.rolling(window=6, min_periods=1).min()
        )
        result[f'{col}_roll_max_6h'] = result.groupby('ICUSTAY_ID')[col].transform(
            lambda x: x.rolling(window=6, min_periods=1).max()
        )
        
        # Rate of change (difference from previous hour)
        result[f'{col}_diff_1h'] = result.groupby('ICUSTAY_ID')[col].diff(1)
        
        # Rate of change (difference from 3 hours ago)
        result[f'{col}_diff_3h'] = result.groupby('ICUSTAY_ID')[col].diff(3)
        
        # Lag features
        result[f'{col}_lag_1h'] = result.groupby('ICUSTAY_ID')[col].shift(1)
        result[f'{col}_lag_3h'] = result.groupby('ICUSTAY_ID')[col].shift(3)
        result[f'{col}_lag_6h'] = result.groupby('ICUSTAY_ID')[col].shift(6)
    
    return result

print("Adding temporal features...")
vitals_featured = add_temporal_features(vitals_imputed)

print(f"\nFeatures added. New shape: {vitals_featured.shape}")
print(f"Total features: {len(vitals_featured.columns)}")

FEATURE ENGINEERING
Adding temporal features...

Features added. New shape: (307210, 72)
Total features: 72


In [32]:
# Add time-based features
print("\nAdding time-based features...")

# Hours since ICU admission (already have this as 'HOUR')
vitals_featured['hours_since_admission'] = vitals_featured['HOUR']

# Normalized time in ICU (0 to 1 scale per patient)
vitals_featured['time_normalized'] = vitals_featured.groupby('ICUSTAY_ID')['HOUR'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1)
)

# Is it early in ICU stay (first 24 hours)?
vitals_featured['is_first_24h'] = (vitals_featured['HOUR'] < 24).astype(int)

print("Time-based features added")


Adding time-based features...
Time-based features added


In [33]:
# Fill NaN values created by rolling/lag operations
print("\nFilling NaN from feature engineering...")

# For rolling features, fill with the first available value
vitals_featured = vitals_featured.groupby('ICUSTAY_ID', group_keys=False).apply(
    lambda x: x.ffill().bfill()
)

# Check for remaining NaN
nan_counts = vitals_featured.isna().sum()
nan_cols = nan_counts[nan_counts > 0]
if len(nan_cols) > 0:
    print(f"Remaining NaN columns: {len(nan_cols)}")
    # Fill with 0 for remaining
    vitals_featured = vitals_featured.fillna(0)
else:
    print("No remaining NaN values")


Filling NaN from feature engineering...
Remaining NaN columns: 38


## 7. Deterioration Labels

In [42]:
print("="*60)
print("DETERIORATION LABELS")
print("="*60)

# Merge mortality information (only if not already merged)
if 'HOSPITAL_EXPIRE_FLAG' not in vitals_featured.columns:
    mortality_info = admissions[['HADM_ID', 'HOSPITAL_EXPIRE_FLAG', 'DEATHTIME']].drop_duplicates()
    vitals_featured = vitals_featured.merge(mortality_info, on='HADM_ID', how='left')

# Add ICU stay info for death time calculation (only if not already merged)
if 'INTIME' not in vitals_featured.columns:
    vitals_featured = vitals_featured.merge(
        icustays[['ICUSTAY_ID', 'INTIME']].drop_duplicates(),
        on='ICUSTAY_ID',
        how='left'
    )

print(f"Patients with hospital mortality: {vitals_featured.groupby('ICUSTAY_ID')['HOSPITAL_EXPIRE_FLAG'].first().mean() * 100:.1f}%")

DETERIORATION LABELS
Patients with hospital mortality: 11.4%


In [43]:
print("="*60)
print("DETERIORATION LABELS")
print("="*60)

# First, check current shape
print(f"\nCurrent vitals_featured shape: {vitals_featured.shape}")

# Merge mortality information - but avoid duplicating rows
mortality_info = admissions[['HADM_ID', 'HOSPITAL_EXPIRE_FLAG', 'DEATHTIME']].drop_duplicates()

# Check if columns already exist (avoid re-merging)
if 'HOSPITAL_EXPIRE_FLAG' not in vitals_featured.columns:
    vitals_featured = vitals_featured.merge(mortality_info, on='HADM_ID', how='left')
else:
    print("Mortality info already merged, skipping...")

# Add ICU stay info for death time calculation
if 'INTIME' not in vitals_featured.columns:
    vitals_featured = vitals_featured.merge(
        icustays[['ICUSTAY_ID', 'INTIME']].drop_duplicates(),
        on='ICUSTAY_ID',
        how='left'
    )
else:
    print("ICU stay info already merged, skipping...")

print(f"Shape after merges: {vitals_featured.shape}")

# Calculate mortality rate CORRECTLY
# Count unique ICU stays with mortality flag = 1
unique_stays = vitals_featured.drop_duplicates(subset=['ICUSTAY_ID'])
mortality_count = unique_stays['HOSPITAL_EXPIRE_FLAG'].sum()
total_stays = len(unique_stays)
mortality_rate = mortality_count / total_stays * 100

print(f"\nMortality Statistics:")
print(f"  Total unique ICU stays: {total_stays:,}")
print(f"  Patients with hospital mortality: {int(mortality_count):,} ({mortality_rate:.1f}%)")

DETERIORATION LABELS

Current vitals_featured shape: (307210, 84)
Mortality info already merged, skipping...
ICU stay info already merged, skipping...
Shape after merges: (307210, 84)

Mortality Statistics:
  Total unique ICU stays: 1,977
  Patients with hospital mortality: 226 (11.4%)


In [44]:
# 3. Vital sign deterioration (threshold-based)
# Flag hours where vitals are in critical ranges

# Define critical thresholds (using normalized values - need to convert)
# For normalized data, we'll use z-scores
# Critical = more than 2 standard deviations from mean

def create_vital_deterioration_flags(df):
    """Create flags for abnormal vital signs"""
    result = df.copy()
    
    # For normalized data: |z| > 2 is concerning
    for col in VITAL_COLS:
        result[f'{col}_critical'] = (abs(result[col]) > 2).astype(int)
    
    # Combined deterioration score (number of critical vitals)
    critical_cols = [f'{col}_critical' for col in VITAL_COLS]
    result['n_critical_vitals'] = result[critical_cols].sum(axis=1)
    
    # Binary deterioration flag (2+ critical vitals)
    result['vital_deterioration'] = (result['n_critical_vitals'] >= 2).astype(int)
    
    return result

vitals_featured = create_vital_deterioration_flags(vitals_featured)

print(f"\nVital sign deterioration:")
print(f"  Hours with 2+ critical vitals: {vitals_featured['vital_deterioration'].sum():,} ({vitals_featured['vital_deterioration'].mean()*100:.2f}%)")


Vital sign deterioration:
  Hours with 2+ critical vitals: 17,919 (5.83%)


## 8. Create Windowed Sequences for Forecasting

In [45]:
print("="*60)
print("WINDOWED SEQUENCE CREATION")
print("="*60)

# Forecasting configuration
# Your proposal: 60-minute input → 240-minute forecast
# With hourly data: 6-hour input → 4-hour (or more) forecast is reasonable

INPUT_HOURS = 6      # 6 hours of historical data
FORECAST_HOURS = 6   # Predict next 6 hours

print(f"\nWindow configuration:")
print(f"  Input window: {INPUT_HOURS} hours")
print(f"  Forecast horizon: {FORECAST_HOURS} hours")
print(f"  Total sequence length: {INPUT_HOURS + FORECAST_HOURS} hours")

WINDOWED SEQUENCE CREATION

Window configuration:
  Input window: 6 hours
  Forecast horizon: 6 hours
  Total sequence length: 12 hours


In [46]:
# CREATINg MORTALITY LABELS before sequence creation

from datetime import timedelta

# 1. Mortality within next 24 hours
def label_mortality_24h(row):
    if pd.isna(row['DEATHTIME']) or pd.isna(row['INTIME']):
        return 0
    try:
        current_time = row['INTIME'] + timedelta(hours=row['HOUR'])
        hours_to_death = (row['DEATHTIME'] - current_time).total_seconds() / 3600
        return 1 if 0 < hours_to_death <= 24 else 0
    except:
        return 0

print("Creating mortality_24h labels...")
vitals_featured['mortality_24h'] = vitals_featured.apply(label_mortality_24h, axis=1)

# 2. Mortality within next 48 hours
def label_mortality_48h(row):
    if pd.isna(row['DEATHTIME']) or pd.isna(row['INTIME']):
        return 0
    try:
        current_time = row['INTIME'] + timedelta(hours=row['HOUR'])
        hours_to_death = (row['DEATHTIME'] - current_time).total_seconds() / 3600
        return 1 if 0 < hours_to_death <= 48 else 0
    except:
        return 0

vitals_featured['mortality_48h'] = vitals_featured.apply(label_mortality_48h, axis=1)

print(f"\nDeterioration label distribution:")
print(f"  Mortality within 24h: {vitals_featured['mortality_24h'].sum():,} ({vitals_featured['mortality_24h'].mean()*100:.2f}%)")
print(f"  Mortality within 48h: {vitals_featured['mortality_48h'].sum():,} ({vitals_featured['mortality_48h'].mean()*100:.2f}%)")

Creating mortality_24h labels...

Deterioration label distribution:
  Mortality within 24h: 3,536 (1.15%)
  Mortality within 48h: 6,854 (2.23%)


In [47]:
# 3. Vital sign deterioration flags
def create_vital_deterioration_flags(df):
    result = df.copy()
    
    for col in VITAL_COLS:
        if col in result.columns:
            result[f'{col}_critical'] = (abs(result[col]) > 2).astype(int)
    
    critical_cols = [f'{col}_critical' for col in VITAL_COLS if f'{col}_critical' in result.columns]
    result['n_critical_vitals'] = result[critical_cols].sum(axis=1)
    result['vital_deterioration'] = (result['n_critical_vitals'] >= 2).astype(int)
    
    return result

vitals_featured = create_vital_deterioration_flags(vitals_featured)

print(f"Vital sign deterioration:")
print(f"  Hours with 2+ critical vitals: {vitals_featured['vital_deterioration'].sum():,} ({vitals_featured['vital_deterioration'].mean()*100:.2f}%)")

Vital sign deterioration:
  Hours with 2+ critical vitals: 17,919 (5.83%)


In [48]:
def create_sequences(df, input_hours, forecast_hours, vital_cols):
    """
    Create input-output sequences for each ICU stay.
    
    Returns:
        X: Input sequences (n_samples, input_hours, n_features)
        y: Target sequences (n_samples, forecast_hours, n_vitals)
        meta: Metadata (ICUSTAY_ID, start_hour, etc.)
    """
    sequences_X = []
    sequences_y = []
    metadata = []
    
    # Features to use as input
    feature_cols = vital_cols + [f'{v}_roll_mean_6h' for v in vital_cols] + \
                   [f'{v}_roll_std_6h' for v in vital_cols] + \
                   [f'{v}_diff_1h' for v in vital_cols] + \
                   ['hours_since_admission', 'time_normalized']
    
    # Filter to only existing columns
    feature_cols = [c for c in feature_cols if c in df.columns]
    
    total_window = input_hours + forecast_hours
    
    for icustay_id in df['ICUSTAY_ID'].unique():
        patient_data = df[df['ICUSTAY_ID'] == icustay_id].sort_values('HOUR')
        
        if len(patient_data) < total_window:
            continue
        
        # Create sliding windows
        for start_idx in range(len(patient_data) - total_window + 1):
            # Input window
            input_window = patient_data.iloc[start_idx:start_idx + input_hours]
            # Target window
            target_window = patient_data.iloc[start_idx + input_hours:start_idx + total_window]
            
            # Extract features and targets
            X = input_window[feature_cols].values
            y = target_window[vital_cols].values
            
            sequences_X.append(X)
            sequences_y.append(y)
            metadata.append({
                'ICUSTAY_ID': icustay_id,
                'start_hour': input_window['HOUR'].iloc[0],
                'mortality_in_window': target_window['mortality_24h'].max()
            })
    
    return np.array(sequences_X), np.array(sequences_y), pd.DataFrame(metadata), feature_cols

print("Creating sequences...")
X, y, meta, feature_names = create_sequences(
    vitals_featured, 
    INPUT_HOURS, 
    FORECAST_HOURS, 
    VITAL_COLS
)

print(f"\nSequences created:")
print(f"  X shape: {X.shape} (samples, input_hours, features)")
print(f"  y shape: {y.shape} (samples, forecast_hours, vitals)")
print(f"  Number of features: {len(feature_names)}")

Creating sequences...

Sequences created:
  X shape: (285482, 6, 26) (samples, input_hours, features)
  y shape: (285482, 6, 6) (samples, forecast_hours, vitals)
  Number of features: 26


In [36]:
# Save feature names for later use
print("\nFeature names:")
for i, name in enumerate(feature_names):
    print(f"  {i}: {name}")


Feature names:
  0: HR
  1: SpO2
  2: SBP
  3: DBP
  4: MBP
  5: RR
  6: Temp
  7: HR_roll_mean_6h
  8: SpO2_roll_mean_6h
  9: SBP_roll_mean_6h
  10: DBP_roll_mean_6h
  11: MBP_roll_mean_6h
  12: RR_roll_mean_6h
  13: Temp_roll_mean_6h
  14: HR_roll_std_6h
  15: SpO2_roll_std_6h
  16: SBP_roll_std_6h
  17: DBP_roll_std_6h
  18: MBP_roll_std_6h
  19: RR_roll_std_6h
  20: Temp_roll_std_6h
  21: HR_diff_1h
  22: SpO2_diff_1h
  23: SBP_diff_1h
  24: DBP_diff_1h
  25: MBP_diff_1h
  26: RR_diff_1h
  27: Temp_diff_1h
  28: hours_since_admission
  29: time_normalized


## 9. Train/Validation/Test Split

In [49]:
print("="*60)
print("TRAIN/VALIDATION/TEST SPLIT")
print("="*60)

# Split by ICU stay (not by individual sequences) to prevent data leakage
unique_stays = meta['ICUSTAY_ID'].unique()
print(f"\nTotal unique ICU stays: {len(unique_stays)}")

# 70% train, 15% validation, 15% test
train_stays, temp_stays = train_test_split(unique_stays, test_size=0.30, random_state=42)
val_stays, test_stays = train_test_split(temp_stays, test_size=0.50, random_state=42)

print(f"  Train stays: {len(train_stays)}")
print(f"  Validation stays: {len(val_stays)}")
print(f"  Test stays: {len(test_stays)}")

TRAIN/VALIDATION/TEST SPLIT

Total unique ICU stays: 1973
  Train stays: 1381
  Validation stays: 296
  Test stays: 296


In [50]:
# Create boolean masks
train_mask = meta['ICUSTAY_ID'].isin(train_stays)
val_mask = meta['ICUSTAY_ID'].isin(val_stays)
test_mask = meta['ICUSTAY_ID'].isin(test_stays)

# Split data
X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]
X_test, y_test = X[test_mask], y[test_mask]

meta_train = meta[train_mask].reset_index(drop=True)
meta_val = meta[val_mask].reset_index(drop=True)
meta_test = meta[test_mask].reset_index(drop=True)

print(f"\nSequence counts:")
print(f"  Train: {len(X_train):,} sequences")
print(f"  Validation: {len(X_val):,} sequences")
print(f"  Test: {len(X_test):,} sequences")


Sequence counts:
  Train: 203,407 sequences
  Validation: 35,151 sequences
  Test: 46,924 sequences


In [51]:
# Verify no data leakage
train_set = set(meta_train['ICUSTAY_ID'])
val_set = set(meta_val['ICUSTAY_ID'])
test_set = set(meta_test['ICUSTAY_ID'])

assert len(train_set & val_set) == 0, "Data leakage: train and val overlap!"
assert len(train_set & test_set) == 0, "Data leakage: train and test overlap!"
assert len(val_set & test_set) == 0, "Data leakage: val and test overlap!"

print("No data leakage detected - splits are clean")

No data leakage detected - splits are clean


## 10. Save Processed Data

In [52]:
print("="*60)
print("SAVING PROCESSED DATA")
print("="*60)

# Save as numpy arrays (for LSTM)
np.save(f"{OUTPUT_PATH}\\X_train.npy", X_train)
np.save(f"{OUTPUT_PATH}\\y_train.npy", y_train)
np.save(f"{OUTPUT_PATH}\\X_val.npy", X_val)
np.save(f"{OUTPUT_PATH}\\y_val.npy", y_val)
np.save(f"{OUTPUT_PATH}\\X_test.npy", X_test)
np.save(f"{OUTPUT_PATH}\\y_test.npy", y_test)

print("Numpy arrays saved")

# Save metadata
meta_train.to_csv(f"{OUTPUT_PATH}\\meta_train.csv", index=False)
meta_val.to_csv(f"{OUTPUT_PATH}\\meta_val.csv", index=False)
meta_test.to_csv(f"{OUTPUT_PATH}\\meta_test.csv", index=False)

print("Metadata saved")

# Save feature names
pd.DataFrame({'feature': feature_names}).to_csv(f"{OUTPUT_PATH}\\feature_names.csv", index=False)
pd.DataFrame({'vital': VITAL_COLS}).to_csv(f"{OUTPUT_PATH}\\vital_names.csv", index=False)

print("Feature names saved")

SAVING PROCESSED DATA
Numpy arrays saved
Metadata saved
Feature names saved


In [53]:
# Save the full processed dataset (for ARIMA and other analyses)
vitals_featured.to_csv(f"{OUTPUT_PATH}\\vitals_processed_full.csv", index=False)
print(f"Full processed dataset saved: {len(vitals_featured):,} rows")

# Save configuration
config = {
    'INPUT_HOURS': INPUT_HOURS,
    'FORECAST_HOURS': FORECAST_HOURS,
    'VITAL_COLS': VITAL_COLS,
    'N_FEATURES': len(feature_names),
    'N_TRAIN': len(X_train),
    'N_VAL': len(X_val),
    'N_TEST': len(X_test)
}

import json
with open(f"{OUTPUT_PATH}\\config.json", 'w') as f:
    json.dump(config, f, indent=2)

print("Configuration saved")

Full processed dataset saved: 307,210 rows
Configuration saved


In [58]:
# Summary
print("\n" + "="*60)
print("PREPROCESSING COMPLETE - SUMMARY")
print("="*60)

print(f"""
Data Configuration:
  - Input window: {INPUT_HOURS} hours
  - Forecast horizon: {FORECAST_HOURS} hours
  - Vital signs: {VITAL_COLS}
  - Total features: {len(feature_names)}

Dataset Sizes:
  - Train: {len(X_train):,} sequences from {len(train_stays)} patients
  - Validation: {len(X_val):,} sequences from {len(val_stays)} patients
  - Test: {len(X_test):,} sequences from {len(test_stays)} patients

Array Shapes:
  - X: (n_samples, {INPUT_HOURS}, {len(feature_names)}) 
  - y: (n_samples, {FORECAST_HOURS}, {len(VITAL_COLS)})

Output Files Saved to: {OUTPUT_PATH}
  - X_train.npy, y_train.npy
  - X_val.npy, y_val.npy  
  - X_test.npy, y_test.npy
  - meta_train.csv, meta_val.csv, meta_test.csv
  - vital_scaler.pkl (for inverse transform)
  - vitals_processed_full.csv (for ARIMA)
  - config.json
""")


PREPROCESSING COMPLETE - SUMMARY

Data Configuration:
  - Input window: 6 hours
  - Forecast horizon: 6 hours
  - Vital signs: ['HR', 'SpO2', 'SBP', 'DBP', 'MBP', 'RR']
  - Total features: 26

Dataset Sizes:
  - Train: 203,407 sequences from 1381 patients
  - Validation: 35,151 sequences from 296 patients
  - Test: 46,924 sequences from 296 patients

Array Shapes:
  - X: (n_samples, 6, 26) 
  - y: (n_samples, 6, 6)

Output Files Saved to: C:\Users\hasin\Downloads\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\processed
  - X_train.npy, y_train.npy
  - X_val.npy, y_val.npy  
  - X_test.npy, y_test.npy
  - meta_train.csv, meta_val.csv, meta_test.csv
  - vital_scaler.pkl (for inverse transform)
  - vitals_processed_full.csv (for ARIMA)
  - config.json



---

## Data Loading Template (for next notebooks)

```python
import numpy as np
import pandas as pd
import pickle
import json

```
## INSERT PATH FROM YOUR DESTINATION here
```python
OUTPUT_PATH = r"C:\Users\hasin\Downloads\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4\processed"

# Load config
with open(f"{OUTPUT_PATH}\\config.json", 'r') as f:
    config = json.load(f)

# Load sequences
X_train = np.load(f"{OUTPUT_PATH}\\X_train.npy")
y_train = np.load(f"{OUTPUT_PATH}\\y_train.npy")
X_val = np.load(f"{OUTPUT_PATH}\\X_val.npy")
y_val = np.load(f"{OUTPUT_PATH}\\y_val.npy")
X_test = np.load(f"{OUTPUT_PATH}\\X_test.npy")
y_test = np.load(f"{OUTPUT_PATH}\\y_test.npy")

# Load scaler (for inverse transform)
with open(f"{OUTPUT_PATH}\\vital_scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)

# Load feature names
feature_names = pd.read_csv(f"{OUTPUT_PATH}\\feature_names.csv")['feature'].tolist()
vital_names = pd.read_csv(f"{OUTPUT_PATH}\\vital_names.csv")['vital'].tolist()

print(f"Loaded: X_train {X_train.shape}, y_train {y_train.shape}")
```